# Dashboard

In [1]:
# !pip install -r requirements.txt

In [2]:
# imports

import dash
from dash import dcc, html
from dash.dependencies import Input, Output

import datetime

from keras.models import Sequential
from keras.layers import LSTM, Dense

import matplotlib.pyplot as plt

import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

import numpy as np

import os
import pandas as pd

from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

import pymysql 

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sqlalchemy import create_engine 

from statsmodels.tsa.arima.model import ARIMA

import warnings

2024-11-09 11:27:03.978126: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Suppress all warnings
warnings.filterwarnings('ignore')

In [4]:
# Get the data from the environment variables
hostname = os.getenv('MYSQLHOST')
port=int(os.getenv('MYSQLPORT'))
username = os.getenv('MYSQLUSR')
password = os.getenv('MYSQLPASS')
database_name = os.getenv('MYSQLDB') 
ca_cert_path = '../ca.pem'

# Create MySQL connection object
connection = pymysql.connect( 
    host=hostname, 
    port=port, 
    user=username, 
    password=password, 
    database=database_name, 
    ssl={'ca': ca_cert_path} 
)

# Create the SQLAlchemy engine for MySQL
engine = create_engine(f"mysql+pymysql://{username}:{password}@{hostname}:{port}/{database_name}")

In [5]:
MONGOURI = os.getenv('MONGOURI')
MONGODB = os.getenv('MONGODB')

client = MongoClient(MONGOURI, server_api=ServerApi('1'))

# Select database by name.
# If database does not exist it will create a new one.
db = client[MONGODB]  

# Collections
arima_coll = db['arima']
lstm_coll = db['lstm']

In [6]:
query = "SELECT ticker FROM Company;"

# Fetch data into a pandas DataFrame
company_df = pd.read_sql(query, engine)

In [7]:
company_df.head()

,ticker
0,AAPL
1,AMT
2,AMZN
3,BA
4,BABA


In [8]:
# Get unique values for the dropdown
company = company_df['ticker']
dropdown_options = [{'label': ticker, 'value': ticker} for ticker in company]

In [9]:
# Initialize the Dash app
app = dash.Dash(__name__)

# Layout of the app
app.layout = html.Div([
    html.H1("Dashboard"),
    html.Label("Select a Company:"),
    dcc.Dropdown(
        id='company-dropdown',
        options=dropdown_options,  
        value=company[0]  
    ), 
    html.H3("ARIMA Model"),
    html.Div(id='arima-1d', style={'margin-top': '20px'}),
    html.Div(id='arima-3d', style={'margin-top': '20px'}),
    html.Div(id='arima-7d', style={'margin-top': '20px'}),

    html.H3("LSTM Model"),
    html.Div(id='lstm-1d', style={'margin-top': '20px'}),
    html.Div(id='lstm-3d', style={'margin-top': '20px'}),
    html.Div(id='lstm-7d', style={'margin-top': '20px'}),

    
])

# Callback to update the output based on user input
@app.callback(
    [Output('arima-1d', 'children'), Output('arima-3d', 'children'), Output('arima-7d', 'children'),
    Output('lstm-1d', 'children'), Output('lstm-3d', 'children'), Output('lstm-7d', 'children')],
    Input('company-dropdown', 'value')
)
def update_output(company):
    query = {"ticker": company}
    projection = {"_id": 0, "ticker": 1, "1D": 1, "3D": 1, "7D": 1}

    # ARIMA
    res = arima_coll.find(query, projection)
    arima = list(res)

    # LSTM
    res = lstm_coll.find(query, projection)
    lstm = list(res)   
    
    return [f"1 day = {arima[0]['1D']}", f"3 days = {arima[0]['3D']}", f"7 days = {arima[0]['7D']}",
           f"1 day = {lstm[0]['1D']}", f"3 days = {lstm[0]['3D']}", f"7 days = {lstm[0]['7D']}"]


# Run the app
if __name__ == '__main__':
    #app.run_server(debug=True)
    app.run_server(mode="inline", port=8051, debug=False)